In [131]:
import torch
from torch import nn
from torch.nn.init import xavier_uniform_
from torch.nn.init import constant_
from torch.nn.init import xavier_normal_
import math
import torch.nn.functional as F
from enum import IntEnum
import numpy as np
import pickle
from akt import AKT
import os
from load_data import DATA, PID_DATA


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'

with open('result/akt_pid/assist2009_pid/args.pkl', 'rb') as f:
    params = pickle.load(f)

In [132]:
model_type = 'akt'
filename = "_b24_nb1_gn-1_lr1e-05_s224_sl200_do0.05_dm256_ts1_kq1_l21e-05_178"
model = AKT(n_question=params.n_question, n_pid=params.n_pid, n_blocks=params.n_block, d_model=params.d_model,
                    dropout=params.dropout, kq_same=params.kq_same, model_type=model_type, l2=params.l2).to(device)

checkpoint = torch.load(os.path.join( 'model', params.model, params.save, filename))
model.load_state_dict(checkpoint['model_state_dict'])


<All keys matched successfully>

In [133]:
params

Namespace(max_iter=300, train_set=1, seed=224, optim='adam', batch_size=24, lr=1e-05, maxgradnorm=-1, final_fc_dim=512, d_model=256, d_ff=1024, dropout=0.05, n_block=1, n_head=8, kq_same=1, l2=1e-05, q_embed_dim=50, qa_embed_dim=256, memory_size=50, init_std=0.1, hidden_dim=512, lamda_r=0.1, lamda_w1=0.1, lamda_w2=0.1, model='akt_pid', dataset='assist2009_pid', n_question=110, seqlen=200, data_dir='data/assist2009_pid', data_name='assist2009_pid', n_pid=16891, save='assist2009_pid', load='assist2009_pid')

In [134]:
train_data_path = "assist2009_pid_example1.csv"
if "pid" not in params.data_name:
    dat = DATA(n_question=params.n_question,
                seqlen=params.seqlen, separate_char=',')
else:
    dat = PID_DATA(n_question=params.n_question,
                    seqlen=params.seqlen, separate_char=',')
    
train_q_data, train_qa_data, train_pid = dat.load_data(train_data_path)
q, qa, pid = train_q_data[:,:], train_qa_data[:,:], train_pid[:,:]
target = qa[:,:]
target = (target - 1) / params.n_question
target_1 = np.floor(target)

In [135]:
q = torch.from_numpy(q).long().to(device)
qa = torch.from_numpy(qa).long().to(device) #qa = q+target*n_question
target = torch.from_numpy(target_1).long().to(device)
pid = torch.from_numpy(pid).long().to(device)

In [136]:
pid.shape

torch.Size([1, 200])

In [137]:
model.eval()
with torch.no_grad():
    loss, pred, ct = model(q,qa,target,pid)

In [138]:
target = target_1.reshape((-1,))
nopadding_index = np.flatnonzero(target >= -0.9)
nopadding_index = nopadding_index.tolist()
pred_nopadding = pred[nopadding_index]
target_nopadding = target[nopadding_index]

In [139]:
pred_nopadding

tensor([0.3372, 0.2478], device='cuda:0')

In [140]:
target_nopadding

array([0., 1.])

In [141]:
history = {'q':[1,3,6],'target':[1,0,1],'pid':[3,5,7]} 
q,target,pid = (np.array(value) for value in history.values())
qa = q+target*13

padded_q = np.zeros((1, 10))
padded_qa = np.zeros((1, 10))
padded_target = np.full((1,10),-1)
padded_pid = np.zeros((1, 10))

padded_q[0, :len(q)] = q
padded_qa[0, :len(q)] = qa
padded_target[0, :len(target)] = target
padded_pid[0, :len(pid)] = pid

q = torch.tensor(padded_q).long().to(device)
qa = torch.tensor(padded_qa).long().to(device)
target = torch.tensor(padded_target).long().to(device)
pid = torch.tensor(padded_pid).long().to(device)


model.eval()
with torch.no_grad():
    loss, pred, ct = model(q,qa,target,pid)
# # target_1 = np.floor(target)
# q = torch.tensor(q+[self.curr_q]).long().to(self.device)
# target = torch.tensor(target_1+[0]).long().to(self.device)
# pid = torch.tensor(pid+[self.curr_pid]).long().to(self.device)
# qa = q+target*self.params.n_question

In [142]:
nopadding_index = np.flatnonzero(padded_target.reshape((-1,)) >= -0.9).tolist()

In [148]:
pred[nopadding_index][-1].item()

0.010884453542530537

tensor(False)

In [121]:
padded_target.reshape((-1,))

array([ 1,  0,  1, -1, -1, -1, -1, -1, -1, -1])

In [116]:
target.reshape((-1,))

array([1, 0, 1])